In [17]:
from langchain_openai import ChatOpenAI
from decouple import config


In [21]:
from langchain_community.llms import Ollama

In [22]:
llm = Ollama(model='llama2')

In [23]:
llm.invoke("how can langsmith help with testing?")

"Langsmith is a tool that can help with testing in several ways:\n\n1. **Testing language models**: Langsmith provides a simple and efficient way to test language models, including those trained on large datasets like BERT or RoBERTa. You can use Langsmith to evaluate the performance of your model on various tasks, such as text classification, sentiment analysis, and question answering.\n2. **Evaluating model performance**: Langsmith allows you to evaluate the performance of your language model using a variety of metrics, including accuracy, F1 score, and ROUGE score. You can also use Langsmith to compare the performance of different models or to analyze the effectiveness of different training strategies.\n3. **Generating test data**: Langsmith can be used to generate test data for your language model, including sentences, paragraphs, or even entire documents. This can help you evaluate your model's ability to generalize to new, unseen data.\n4. **Hyperparameter tuning**: Langsmith pro

In [24]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [25]:
chain = prompt | llm 

In [26]:
chain.invoke({"input": "how can langsmith help with testing?"})

'As a world-class technical documentation writer, I can provide valuable insights and suggestions for testing and quality assurance in the following ways:\n\n1. Developing Test Plans: Langsmith can assist in creating comprehensive test plans that cover all aspects of the software or system being documented. This includes identifying the scope of testing, determining the appropriate testing methodologies and tools, and creating test cases that ensure thorough coverage.\n2. Writing Test Cases: I can help write test cases based on the requirements and specifications of the system or software being tested. These test cases can be used to verify that the system functions as intended and to identify any bugs or issues.\n3. Conducting Regression Testing: Langsmith can assist in conducting regression testing, which involves re-executing a series of tests to ensure that changes made to the software or system have not introduced new bugs or affected existing functionality.\n4. Creating Test Docu

In [27]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [28]:
chain = prompt | llm | output_parser


In [29]:
chain.invoke({"input": "how can langsmith help with testing?"})

'As a world-class technical documentation writer, I must say that Langsmith is an excellent tool for testing purposes. Here are some ways in which Langsmith can help with testing:\n\n1. **Automated Content Generation**: With Langsmith, you can generate high-quality content automatically, which can save time and effort during the testing process. You can use Langsmith to create test cases, test scripts, and even entire documentation sets, all of which can be tailored to your specific testing needs.\n2. **Content Customization**: Langsmith allows you to customize content to fit your specific testing requirements. You can use the tool to generate content that is tailored to your testing environment, including the specific devices, platforms, and systems you are testing.\n3. **Consistency**: One of the biggest challenges in testing is ensuring consistency across different test cases and test scripts. Langsmith helps to address this challenge by generating content that is consistent and fol

In [1]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

docs = loader.load()

In [5]:
# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings()

In [30]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [31]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [32]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [33]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'\nBased on the provided context, langsmith can help with testing by allowing users to visualize their test results.'

In [34]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [35]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

Based on the provided context, LangSmith can help with testing by providing features such as:

* Evaluation capabilities: LangSmith offers evaluation capabilities that allow you to test and assess the performance of your LLM applications.
* Tracing capabilities: LangSmith provides tracing capabilities that enable you to track the behavior of your LLM applications during testing, allowing you to identify potential issues and optimize their performance.
* Integration with LangChain: LangSmith seamlessly integrates with LangChain, the go-to open source framework for building LLMs, providing a comprehensive platform for testing and evaluating LLM applications.


In [36]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [37]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.Setup: Learn 

In [38]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [39]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following s

In [40]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [8]:
llm = ChatOpenAI(openai_api_key=config('openai_api_key'))

In [9]:
llm.invoke("how can langsmith help with testing?")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [1]:
llm.invoke("how can langsmith help with testing?")

NameError: name 'llm' is not defined